In [1]:
#
#	Wenbo Li 03/29/2022
#
#	env:	Julia 1.7.2,    Python 3.9.5,   Pytorch 1.10.2,   CUDA 11.3,   
#       	cuDNN 8.2.0,    AMD Ryzen-3950X(numThreads=32),    Nvidia RTX-2080Ti,  Windows 10

# Setting up environment

In [2]:
using PyCall
torch = pyimport("torch")
np = pyimport("numpy")
py"""
import torch
import numpy as np
from scipy.ndimage import distance_transform_edt as distance
from scipy.ndimage import _nd_image
"""
device = torch.device("cuda:0");
println(device)

let
	using Pkg
	Pkg.activate(mktempdir())
	Pkg.Registry.update()
	Pkg.add("BenchmarkTools")
	Pkg.add("Plots")
	Pkg.add("PlutoUI")
	Pkg.add("DataFrames")
	Pkg.add("CSV")
	Pkg.add("CUDA")
	Pkg.add(url="https://github.com/Dale-Black/DistanceTransforms.jl")
	Pkg.add(url="https://github.com/Dale-Black/Losers.jl");
end

using PlutoUI
using BenchmarkTools
using Plots
using DataFrames
using CSV
using CUDA
using DistanceTransforms
using Losers;

In [ ]:
# init arrays
hd_mean_cpu_python = []
hd_std_cpu_python = []

hd_mean_cpu_julia = []
hd_std_cpu_julia = []

hd_mean_gpu_python = []
hd_std_gpu_python = []

hd_mean_gpu_julia = []
hd_std_gpu_julia = []

dice_mean_cpu_python = []
dice_std_cpu_python = []

dice_mean_cpu_julia = []
dice_std_cpu_julia = []

dice_mean_gpu_python = []
dice_std_gpu_python = []

dice_mean_gpu_julia = []
dice_std_gpu_julia = []

edt_mean_cpu_python = []
edt_std_cpu_python = []

edt_mean_cpu_julia = []
edt_std_cpu_julia = []

sedt_mean_cpu_julia = []
sedt_std_cpu_julia = []

sedtP_mean_cpu_julia = []
sedtP_std_cpu_julia = []

sedtGPU_mean_gpu_julia = []
sedtGPU_std_gpu_julia = []

finish_size=1000;

# 1.1 Hausdorff Loss

In [ ]:
#define hd_loss for python
py"""
def hd_loss(seg_soft, gt, seg_dtm, gt_dtm):
    delta_s = (seg_soft - gt) ** 2
    s_dtm = seg_dtm ** 2
    g_dtm = gt_dtm ** 2
    dtm = s_dtm + g_dtm
    multipled = torch.einsum('xy, xy->xy', delta_s, dtm)
    hd_loss = multipled.mean()
    return hd_loss
"""

In [ ]:
#Hausdorff Loss Python CPU
for n in 2:100:finish_size
	x1= torch.randint(0,2,(n,n))
	tfm1 = torch.from_numpy(py"distance"(x1))
	hd1 = @benchmark py"hd_loss"($x1, $x1, $tfm1, $tfm1)
    push!(hd_mean_cpu_python, BenchmarkTools.mean(hd1).time)
	push!(hd_std_cpu_python, BenchmarkTools.std(hd1).time)
end

In [32]:
#Hausdorff Loss Julia CPU
for n in 2:100:finish_size
    x2 = DistanceTransforms.boolean_indicator(rand([0, 1], n, n))
	tfm2 = DistanceTransforms.SquaredEuclidean(x2)
	x2_dtm = DistanceTransforms.transform(x2, tfm2)
	hd2 = @benchmark hausdorff($x2, $x2, $x2_dtm, $x2_dtm)
		
	push!(hd_mean_cpu_julia, BenchmarkTools.mean(hd2).time)
	push!(hd_std_cpu_julia, BenchmarkTools.std(hd2).time)
end

In [ ]:
#Hausdorff Loss Python GPU
for n in 2:100:finish_size
    x4 = torch.randint(0,2,(n,n)).to(device)
    tfm4 = torch.from_numpy(py"distance"(x4.cpu().numpy())).to(device)
	hd4 = @benchmark py"hd_loss"($x4, $x4, $tfm4, $tfm4)
    
    push!(hd_mean_gpu_python, BenchmarkTools.mean(hd4).time)
	push!(hd_std_gpu_python, BenchmarkTools.std(hd4).time)
end

In [ ]:
#Hausdorff Loss Julia GPU
for n in 2:100:finish_size
    x3 = DistanceTransforms.boolean_indicator(CUDA.rand(n, n))
	dt3 = CuArray{Float32}(undef, size(x3))
	v3 = CUDA.ones(Int64, size(x3))
	z3 = CUDA.zeros(Float32, size(x3) .+ 1)
	tfm3 = DistanceTransforms.SquaredEuclidean(x3, dt3, v3, z3)
	x3_dtm = DistanceTransforms.transform!(x3, tfm3)
	hd3 = @benchmark hausdorff($x3, $x3, $x3_dtm, $x3_dtm)
		
	push!(hd_mean_gpu_julia, BenchmarkTools.mean(hd3).time)
	push!(hd_std_gpu_julia, BenchmarkTools.std(hd3).time)
end

# 1.2 DICE Loss

In [ ]:
#define dice_loss for python
py"""
def dice_loss(score, target):
    target = target.float()
    smooth = 1e-5
    intersect = torch.sum(score * target)
    y_sum = torch.sum(target * target)
    z_sum = torch.sum(score * score)
    loss = (2 * intersect + smooth) / (z_sum + y_sum + smooth)
    loss = 1 - loss
    return loss
"""

In [ ]:
#Dice Loss Python CPU
for n in 2:100:finish_size
    x1 = torch.randint(0,2,(n,n))
    dice1 = @benchmark py"dice_loss"($x1, $x1)
    
    push!(dice_mean_cpu_python, BenchmarkTools.mean(dice1).time)
	push!(dice_std_cpu_python, BenchmarkTools.std(dice1).time)
end

In [ ]:
#Dice Loss Julia CPU
for n in 2:100:finish_size
    x2 = rand([0, 1], n, n)
	dice2 = @benchmark dice($x2, $x2)
		
	push!(dice_mean_cpu_julia, BenchmarkTools.mean(dice2).time)
	push!(dice_std_cpu_julia, BenchmarkTools.std(dice2).time)
end

In [ ]:
#Dice Loss Python GPU
for n in 2:100:finish_size
    x3 = torch.randint(0,2,(n,n)).to(device)
    dice3 = @benchmark py"dice_loss"($x3, $x3)
    
    push!(dice_mean_gpu_python, BenchmarkTools.mean(dice3).time)
	push!(dice_std_gpu_python, BenchmarkTools.std(dice3).time)
end

In [ ]:
#Dice Loss Julia GPU
for n in 2:100:finish_size
    x4 = rand([0, 1], n, n)
	x4 = CuArray(x4)
	dice4 = @benchmark dice($x4, $x4)
		
	push!(dice_mean_gpu_julia, BenchmarkTools.mean(dice4).time)
	push!(dice_std_gpu_julia, BenchmarkTools.std(dice4).time)
end

# 2.1 Euclidean DT

In [ ]:
#Euclidean DT python CPU
for n in 2:100:finish_size
    x1 = torch.randint(0,2,(n,n))
    edt1 = @benchmark torch.from_numpy(py"distance"($x1))
    
    push!(edt_mean_cpu_python, BenchmarkTools.mean(edt1).time)
	push!(edt_std_cpu_python, BenchmarkTools.std(edt1).time)
end

In [ ]:
#Euclidean DT julia CPU
for n in 2:100:finish_size
    x2 = Bool.(rand([0, 1], n, n))
    edt2 = @benchmark euclidean($x2)

    push!(edt_mean_cpu_julia, BenchmarkTools.mean(edt2).time)
    push!(edt_std_cpu_julia, BenchmarkTools.std(edt2).time)
end

# 2.2 Squared Euclidean DT

In [ ]:
#Squared Euclidean DT julia CPU
for n in 2:100:finish_size
    x3 = DistanceTransforms.boolean_indicator(rand([0, 1], n, n))
    tfm3 = DistanceTransforms.SquaredEuclidean(x3)
    sedt = @benchmark DistanceTransforms.transform($x3, $tfm3)
    
    push!(sedt_mean_cpu_julia, BenchmarkTools.mean(sedt).time)
    push!(sedt_std_cpu_julia, BenchmarkTools.std(sedt).time)
end

# 2.3 Squared Euclidean Threaded DT

In [ ]:
#Squared Euclidean Threaded DT julia CPU
for n in 2:100:finish_size
    x4 = DistanceTransforms.boolean_indicator(rand([0, 1], n, n))
    tfm4 = DistanceTransforms.SquaredEuclidean(x4)
    nthreads = Threads.nthreads()
    print("num of nthreads: ")
    print(nthreads)
    sedtP = @benchmark DistanceTransforms.transform!($x4, $tfm4, $nthreads)
    
    push!(sedtP_mean_cpu_julia, BenchmarkTools.mean(sedtP).time)
    push!(sedtP_std_cpu_julia, BenchmarkTools.std(sedtP).time)
end

# 2.4 Squared Euclidean GPU DT

In [ ]:
#Squared Euclidean GPU DT julia GPU
for n in 2:100:finish_size
    x5 = DistanceTransforms.boolean_indicator(CUDA.rand(n, n))
    dt5 = CuArray{Float32}(undef, size(x5))
    v5 = CUDA.ones(Int64, size(x5))
    z5 = CUDA.zeros(Float32, size(x5) .+ 1)
    tfm5 = DistanceTransforms.SquaredEuclidean(x5, dt5, v5, z5)
    sedtGPU = @benchmark DistanceTransforms.transform!($x5, $tfm5)

	push!(sedtGPU_mean_gpu_julia, BenchmarkTools.mean(sedtGPU).time)
	push!(sedtGPU_std_gpu_julia, BenchmarkTools.std(sedtGPU).time)
end

# Saving as csv

In [ ]:
df = DataFrame(
	hd_mean_cpu_python = hd_mean_cpu_python,
	hd_mean_cpu_julia = hd_mean_cpu_julia,
	hd_mean_gpu_python = hd_mean_gpu_python,
	hd_mean_gpu_julia = hd_mean_gpu_julia,
	dice_mean_cpu_python = dice_mean_cpu_python,
	dice_mean_cpu_julia=dice_mean_cpu_julia,
	dice_mean_gpu_python=dice_mean_gpu_python,
	dice_mean_gpu_julia=dice_mean_gpu_julia,
	edt_mean_cpu_python=edt_mean_cpu_python,
	edt_mean_cpu_julia=edt_mean_cpu_julia,
	sedt_mean_cpu_julia=sedt_mean_cpu_julia,
	sedtP_mean_cpu_julia=sedtP_mean_cpu_julia,
	sedtGPU_mean_gpu_julia=sedtGPU_mean_gpu_julia,


	hd_std_cpu_python = hd_std_cpu_python,
	hd_std_cpu_julia = hd_std_cpu_julia,
	hd_std_gpu_python = hd_std_gpu_python,
	hd_std_gpu_julia = hd_std_gpu_julia,
	dice_std_cpu_python = dice_std_cpu_python,
	dice_std_cpu_julia=dice_std_cpu_julia,
	dice_std_gpu_python=dice_std_gpu_python,
	dice_std_gpu_julia=dice_std_gpu_julia,
	edt_std_cpu_python=edt_std_cpu_python,
	edt_std_cpu_julia=edt_std_cpu_julia,
	sedt_std_cpu_julia=sedt_std_cpu_julia,
	sedtP_std_cpu_julia=sedtP_std_cpu_julia,
	sedtGPU_std_gpu_julia=sedtGPU_std_gpu_julia
	)

	path = raw"G:\molloi-lab\project-distance-transforms\performance\pureJulia_pyCallingPython_Losses_DTs_ver1.csv"
	CSV.write(path, df)